# `dtype` konvertieren

Manchmal passen die pandas-Datentypen nicht wirklich gut. Dies kann z.B. auf Serialisierungsformate zurückzuführen sein, die keine Typinformationen enthalten. Manchmal solltet ihr jedoch den Typ auch ändern, um eine bessere Performance zu erzielen – entweder  mehr Manipulationsmöglichkeiten oder weniger Speicherbedarf. In den folgenden Beispielen werden wir verschiedene Konvertierungen einer `Series` vornehmen:

In [1]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.default_rng()
s = pd.Series(rng.normal(size=7))

In [3]:
s

0   -1.507354
1   -1.457442
2   -0.293281
3   -2.854128
4    3.386454
5   -0.003893
6   -1.569178
dtype: float64

## Automatische Konvertierung

[pandas.Series.convert_dtypes](https://pandas.pydata.org/docs/reference/api/pandas.Series.convert_dtypes.html) versucht, eine `Series` in einen Typ zu konvertieren, der `NA` unterstützt. Im Fall unserer `Series` wird der Typ von `float64` in `Float64` geändert:

In [4]:
s.convert_dtypes()

0   -1.507354
1   -1.457442
2   -0.293281
3   -2.854128
4    3.386454
5   -0.003893
6   -1.569178
dtype: Float64

Bedauerlicherweise habe ich jedoch mit `convert_dtypes` kaum Kontrolle darüber, in welchen Datentyp konvertiert wird. Daher bevorzuge ich [pandas.Series.astype](https://pandas.pydata.org/docs/reference/api/pandas.Series.astype.html):

In [5]:
s.astype('Float32')

0   -1.507354
1   -1.457442
2   -0.293281
3   -2.854128
4    3.386455
5   -0.003893
6   -1.569178
dtype: Float32

Die Verwendung des richtigen Typs kann Speicherplatz einsparen. Üblich ist ein 8 Byte breiter Datentyp, also `int64` oder `float64`. Wenn ihr einen schmaleren Typ verwenden könnt, reduziert dies den Speicherverbrauch deutlich, sodass ihr mehr Daten verarbeiten könnt. Ihr könnt NumPy verwenden, um die Grenzen von Integer- und Float-Typen zu überprüfen:

In [6]:
np.iinfo('int64')

iinfo(min=-9223372036854775808, max=9223372036854775807, dtype=int64)

In [7]:
np.finfo('float32')

finfo(resolution=1e-06, min=-3.4028235e+38, max=3.4028235e+38, dtype=float32)

In [8]:
np.finfo('float64')

finfo(resolution=1e-15, min=-1.7976931348623157e+308, max=1.7976931348623157e+308, dtype=float64)

## Speicherverbrauch

Um den Speicherverbrauch der `Series` zu berechnen, könnt ihr [pandas.Series.nbytes](https://pandas.pydata.org/docs/reference/api/pandas.Series.nbytes.html) verwenden um den Speicher, der von den Daten verwendet wird, zu ermitteln. [pandas.Series.memory_usage](https://pandas.pydata.org/docs/reference/api/pandas.Series.memory_usage.html) erfasst darüberhinaus auch den Indexspeicher und den Datentyp. Mit `deep=True` lässt sich auch der Speicherverbrauch auf Systemebene ermitteln.

In [9]:
s.nbytes

56

In [10]:
s.astype('Float32').nbytes

35

In [11]:
s.memory_usage()

184

In [12]:
s.astype('Float32').memory_usage()

163

In [13]:
s.memory_usage(deep=True)

184

## String- und Kategorietypen

Die Methode [pandas.Series.astype](https://pandas.pydata.org/docs/reference/api/pandas.Series.astype.html) kann auch numerische Reihen in Zeichenketten umwandeln, wenn ihr `str` übergebt. Beachtet den `dtype` im folgenden Beispiel:

In [14]:
s.astype(str)

0       -1.5073537736810905
1       -1.4574417599451843
2       -0.2932809096790177
3       -2.8541276191481035
4        3.3864544770215455
5    -0.0038929275758171882
6       -1.5691779561122878
dtype: object

In [15]:
s.astype(str).memory_usage()

184

In [16]:
s.astype(str).memory_usage(deep=True)

662

Zur Konvertierung in einen kategorialen Typ könnt ihr `'category'` als Typ übergeben:

In [17]:
s.astype(str).astype('category')

0       -1.5073537736810905
1       -1.4574417599451843
2       -0.2932809096790177
3       -2.8541276191481035
4        3.3864544770215455
5    -0.0038929275758171882
6       -1.5691779561122878
dtype: category
Categories (7, object): ['-0.0038929275758171882', '-0.2932809096790177', '-1.4574417599451843', '-1.5073537736810905', '-1.5691779561122878', '-2.8541276191481035', '3.3864544770215455']

Eine kategoriale `Series` ist nützlich für String-Daten und kann zu großen Speichereinsparungen führen. Das liegt daran, dass pandas bei der Konvertierung in kategoriale Daten nicht länger Python-Strings für jeden Wert verwendet, sondern sich wiederholende Werte nicht dupliziert werden. Ihr habt immer noch alle Funktionen des `str`-Attributs, aber ihr spart viel Speicherplatz wenn ihr viele doppelte Werte habt und steigert die Leistung, da ihr nicht so viele String-Operationen durchführen müsst.

In [18]:
s.astype('category').memory_usage(deep=True)

491

## Geordnete Kategorien

Um geordnete Kategorien zu erstellen, müsst ihr einen eigenen [pandas.CategoricalDtype](https://pandas.pydata.org/docs/reference/api/pandas.CategoricalDtype.html) definieren:

In [19]:
from pandas.api.types import CategoricalDtype

sorted = pd.Series(sorted(set(s)))
cat_dtype = CategoricalDtype(
    categories=sorted, ordered=True)

s.astype(cat_dtype)

/srv/jupyter/spack/var/spack/environments/python-38/.spack-env/view/lib/python3.8/site-packages/pandas/io/formats/format.py:1429: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


0   -1.507354
1   -1.457442
2   -0.293281
3   -2.854128
4    3.386454
5   -0.003893
6   -1.569178
dtype: category
Categories (7, float64): [-2.854128 < -1.569178 < -1.507354 < -1.457442 < -0.293281 < -0.003893 < 3.386454]

In [20]:
s.astype(cat_dtype).memory_usage(deep=True)

491

In der folgenden Tabelle sind die Typen aufgeführt, die ihr an `astype` übergeben könnt.

Datentyp | Beschreibung
:------- | :-----------
`str`, `'str'` | in Python-String konvertieren
`'string'` | in Pandas-String konvertieren mit `pandas.NA`
`int`, `'int'`, `'int64'` | in NumPy `int64` konvertieren
`'int32'`, `'uint32'` | in NumPy `int32` konvertieren
`'Int64'` | in pandas `Int64` konvertieren mit `pandas.NA`
`float`, `'float'`, `'float64'` | in Floats konvertieren
`'category'` | in `CategoricalDtype` konvertieren mit `pandas.NA`

## Umwandlung in andere Datentypen

Die Methode [pandas.Series.to_numpy](https://pandas.pydata.org/docs/reference/api/pandas.Series.to_numpy.html) oder die Eigenschaft [pandas.Series.values](https://pandas.pydata.org/docs/reference/api/pandas.Series.values.html) liefert uns ein NumPy-Array mit Werten, und [pandas.Series.to_list](https://pandas.pydata.org/docs/reference/api/pandas.Series.to_list.html) gibt eine Python-Liste mit Werten zurück. Warum solltet ihr das wollen? pandas-Objekte sind meist viel benutzerfreundlicher und der Code lässt sich leichter lesen. Zudem werden Python-Listen sehr viel langsamer verarbeitet werden können. Mit [pandas.Series.to_frame](https://pandas.pydata.org/docs/reference/api/pandas.Series.to_frame.html) könnt ihr ggf. einen DataFrame mit einer einzigen Spalte erzeugen:

In [21]:
s.to_frame()

,0
0,-1.507354
1,-1.457442
2,-0.293281
3,-2.854128
4,3.386454
5,-0.003893
6,-1.569178


Auch die Funktion [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html) kann hilfreich sein um in pandas um Werte in Datum und Uhrzeit zu konvertieren.